# my pathetic `List.sequenceResultM` trick

The reasoning—yes, _reasoning_—behind my use of `List.sequenceResultM` [📖 [docs](https://demystifyfp.gitbook.io/fstoolkit-errorhandling/list/sequenceresultm)] is because I really did not understand the monadic purpose for Computation Expressions. The first clue for me was the _M_ in `List.sequenceResultM` from `FsToolkit.ErrorHandling`. To save time by not reading this, take a look at the [Fleece](https://github.com/fsprojects/Fleece) project as Fleece addresses the issues brought up here but is only subset of `FsToolkit.ErrorHandling`.

Given a list of `Result<_,_>`, `List.sequenceResultM` would return the first `Result<_,_>.Error`. This list of results can be a the `OK` values of the properties of some type. I understood that I could use `FsToolkit.ErrorHandling` to check all of these values before attempting to return an `OK` instance of this type.

In [1]:
#!fsharp

#r "nuget:FsToolkit.ErrorHandling"

open System.Text.Json

open FsToolkit.ErrorHandling
open FsToolkit.ErrorHandling.Operator.Result

Installed Packages FsToolkit.ErrorHandling, 4.6.0

## my JSON-result functions

The list of results desired would come from the values stored in JSON. Like an animal, I prefer to traverse my JSON input with functions like the ones below:

In [2]:
#!fsharp

let resultError (elementName: string) =
    Error <| JsonException $"the expected `{elementName}` element is not here."

let toResultFromJsonElement (isKind: JsonValueKind -> bool) doOk (result: Result<JsonElement,JsonException>) =
    match result with
    | Error ex -> Error ex
    | Ok el when el.ValueKind |> isKind -> el |> doOk |> Ok
    | Ok el -> Error <| JsonException($"The expected {nameof(JsonValueKind)} is not here: {el.ValueKind}")

let toResultFromBooleanElement doOk (result: Result<JsonElement,JsonException>) =
    toResultFromJsonElement (fun kind -> kind = JsonValueKind.True || kind = JsonValueKind.False) doOk result

let toResultFromNumericElement doOk (result: Result<JsonElement,JsonException>) =
    toResultFromJsonElement (fun kind -> kind = JsonValueKind.Number) doOk result

let toResultFromStringElement doOk (result: Result<JsonElement,JsonException>) =
    toResultFromJsonElement (fun kind -> kind = JsonValueKind.String) doOk result

let tryGetProperty (elementName: string) (element: JsonElement) =
    if element.ValueKind <> JsonValueKind.Object then
        resultError elementName
    else
        match element.TryGetProperty elementName with
        | false, _ -> resultError elementName
        | true, el -> Ok el

let tryGetRootElement (rawDocument: string) =
        try
            let document = rawDocument |> JsonDocument.Parse
            Ok document.RootElement
        with | exn -> Error <| JsonException(exn.Message, exn)

The functions above are based on the one in my dedicated utility, `JsonDocumentUtility` [[GitHub](https://github.com/BryanWilhite/Songhay.Modules/blob/main/Songhay.Modules/JsonDocumentUtility.fs)].

## the JSON input

With the functions above we can obtain the `rootElementResult` below:

In [3]:
#!fsharp

let rootElementResult =
    @"
        {
            ""one"": 1,
            ""two"": true,
            ""three"": ""is third"",
            ""four"" : {
                ""forty-one"": 41
            }
        }
    "
    |> tryGetRootElement

rootElementResult

Microsoft.FSharp.Core.FSharpResult`2[System.Text.Json.JsonElement,System.Text.Json.JsonException] ResultValue {
 "one": 1,
 "two": true,
 "three": "is third",
 "four" : {
 "forty-one": 41
 }
 } one 1 two true three "is third" four {
 "forty-one": 41
 } forty-one 41 ErrorValue <null>

# the pathetic way to be `OK`

Before I knew about the `result` Computation Expression in `FsToolkit.ErrorHandling`, I would try to get a domain type like this:

In [4]:
#!fsharp

let oneResult = rootElementResult |> Result.bind (tryGetProperty "one")
let twoResult = rootElementResult |> Result.bind (tryGetProperty "two")
let threeResult = rootElementResult |> Result.bind (tryGetProperty "three")
let fortyOneResult =
    rootElementResult
    |> Result.bind (tryGetProperty "four")
    |> Result.bind (tryGetProperty "forty-one")

[
    oneResult
    twoResult
    threeResult
    fortyOneResult
]
|> List.sequenceResultM
|> Result.map (fun _ ->
    {|
        one = oneResult |> toResultFromNumericElement (fun el -> el.GetInt32()) |> Result.valueOr raise
        two = twoResult |> toResultFromBooleanElement (fun el -> el.GetBoolean()) |> Result.valueOr raise
        three = threeResult |> toResultFromStringElement (fun el -> el.GetString()) |> Result.valueOr raise
        fortyOne = fortyOneResult |> toResultFromNumericElement (fun el -> el.GetInt32()) |> Result.valueOr raise
    |})

Microsoft.FSharp.Core.FSharpResult`2[<>f__AnonymousType4017644820`4[System.Int32,System.Int32,System.String,System.Boolean],System.Text.Json.JsonException] ResultValue { fortyOne = 41\n one = 1\n three = "is third"\n two = true } fortyOne 41 one 1 three is third two True ErrorValue <null>

The clarity of my ignorance is impressive. My primitive use of `List.sequenceResultM` is _exactly_ what members behind the result Computation Expression are doing.

## the intended way to be `OK`

The `let!` and `and!` bindings are doing exactly what my use of `List.sequenceResultM` is doing above. However, the clarity of the Computation Expression is far superior:

In [5]:
#!fsharp

result {
    let! one = 
        rootElementResult
        |> Result.bind (tryGetProperty "one")
        |> toResultFromNumericElement (fun el -> el.GetInt32())

    and! two =
        rootElementResult
        |> Result.bind (tryGetProperty "two")
        |> toResultFromBooleanElement (fun el -> el.GetBoolean())

    and! three =
        rootElementResult
        |> Result.bind (tryGetProperty "three")
        |> toResultFromStringElement (fun el -> el.GetString())

    and! fortyOne =
        rootElementResult
        |> Result.bind (tryGetProperty "four")
        |> Result.bind (tryGetProperty "forty-one")
        |> toResultFromNumericElement (fun el -> el.GetInt32())

    return
        {|
            one = one
            two = two
            three = three
            fortyOne = fortyOne
        |}
}

Microsoft.FSharp.Core.FSharpResult`2[<>f__AnonymousType4017644820`4[System.Int32,System.Int32,System.String,System.Boolean],System.Text.Json.JsonException] ResultValue { fortyOne = 41\n one = 1\n three = "is third"\n two = true } fortyOne 41 one 1 three is third two True ErrorValue <null>

## replacing `Result.bind` with the `>>=` operator

To make the Computation Expression above even more readable, we can take advantage of my opening of `FsToolkit.ErrorHandling.Operator.Result` at the beginning of this notebook:

In [6]:
#!fsharp

result {
    let! one = 
        rootElementResult
        >>= (tryGetProperty "one")
        |> toResultFromNumericElement (fun el -> el.GetInt32())

    and! two =
        rootElementResult
        >>= (tryGetProperty "two")
        |> toResultFromBooleanElement (fun el -> el.GetBoolean())

    and! three =
        rootElementResult
        >>= (tryGetProperty "three")
        |> toResultFromStringElement (fun el -> el.GetString())

    and! fortyOne =
        rootElementResult
        >>= (tryGetProperty "four")
        >>= (tryGetProperty "forty-one")
        |> toResultFromNumericElement (fun el -> el.GetInt32())

    return

        {|
            one = one
            two = two
            three = three
            fortyOne = fortyOne
        |}
}

Microsoft.FSharp.Core.FSharpResult`2[<>f__AnonymousType4017644820`4[System.Int32,System.Int32,System.String,System.Boolean],System.Text.Json.JsonException] ResultValue { fortyOne = 41\n one = 1\n three = "is third"\n two = true } fortyOne 41 one 1 three is third two True ErrorValue <null>

## <!-- -->

@[BryanWilhite](https://twitter.com/BryanWilhite)